In [4]:
print_imports()
#!conda env list

Loaded modules:
Numpy                np              1.14.0
Pandas               pd              0.22.0
Keras                ks              2.1.3
Matplotlib           NOT LOADED      No version specified
matplotlib.pyplot    NOT LOADED      No version specified
matplotlib.image     NOT LOADED      No version specified
PIL                  NOT LOADED      No version specified
ExergyUtilities      exergy          2.0.
pyspark              pyspark         2.2.0


In [6]:
#import pyspark
sc = pyspark.SparkContext('local[*]')

ValueError: Cannot run multiple SparkContexts at once; existing SparkContext(app=pyspark-shell, master=local[*]) created by __init__ at <ipython-input-1-9ae820830206>:2 

In [7]:
!rm -rf metastore_db/*.lck

from pyspark.sql import SQLContext
sqlc = SQLContext(sc)

### Step 1
- Load the train and test sets
- Check the schema, the variables have their right types?
- If not, how to correctly load the datasets?

In [21]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType

customSchema = StructType([StructField("PassengerId", IntegerType(), True),
                           StructField("Survived", DoubleType(), True),
                           StructField("Pclass", IntegerType(), True), 
                           StructField("Name", StringType(), True),
                           StructField("Sex", StringType(), True),
                           StructField("Age", DoubleType(), True),
                           StructField("SibSp", IntegerType(), True),
                           StructField("Parch", IntegerType(), True),
                           StructField("Ticket", StringType(), True),
                           StructField("Fare", DoubleType(), True),
                           StructField("Cabin", StringType(), True),
                           StructField("Embarked", StringType(), True)])

customSchema2 = StructType([StructField("PassengerId", IntegerType(), True),
                           StructField("Pclass", IntegerType(), True), 
                           StructField("Name", StringType(), True),
                           StructField("Sex", StringType(), True),
                           StructField("Age", DoubleType(), True),
                           StructField("SibSp", IntegerType(), True),
                           StructField("Parch", IntegerType(), True),
                           StructField("Ticket", StringType(), True),
                           StructField("Fare", DoubleType(), True),
                           StructField("Cabin", StringType(), True),
                           StructField("Embarked", StringType(), True)])

train = sqlc.read.csv("train.csv", header=True, schema=customSchema)
test = sqlc.read.csv("test.csv", header=True, schema=customSchema2)
#train.describe().toPandas()

In [22]:
train.toPandas().head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0.0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,None,S
1,2,1.0,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1.0,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,None,S
3,4,1.0,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0.0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,None,S


### Step 2
- Explore the features of your dataset
- You can use DataFrame's ***describe*** method to get summary statistics
    - hint: ***toPandas*** may be useful to ease the manipulation of small dataframes
- Are there any ***NaN*** values in your dataset?
- If so, define value/values to fill these ***NaN*** values
    - hint: ***na*** property of DataFrames provide several methods of handling NA values

In [23]:
# Calculating summary statistics and turning it into Pandas DF
train_desc = train.describe().toPandas().set_index('summary')
train_desc

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
summary,,,,,,,,,,,,
count,891,891,891,891,891,714,891,891,891,891,204,889
mean,446.0,0.3838383838383838,2.308641975308642,None,None,29.69911764705882,0.5230078563411896,0.38159371492704824,260318.54916792738,32.2042079685746,None,None
stddev,257.3538420152301,0.48659245426485753,0.8360712409770491,None,None,14.526497332334035,1.1027434322934315,0.8060572211299488,471609.26868834975,49.69342859718089,None,None
min,1,0.0,1,"""Andersson, Mr. August Edvard (""""Wennerstrom"""")""",female,0.42,0,0,110152,0.0,A10,C
max,891,1.0,3,"van Melkebeke, Mr. Philemon",male,80.0,8,6,WE/P 5735,512.3292,T,S


In [24]:
train.groupBy('Embarked').count().show()

+--------+-----+
|Embarked|count|
+--------+-----+
|       Q|   77|
|    null|    2|
|       C|  168|
|       S|  644|
+--------+-----+



In [25]:
# Computing correlations between Survived and some features
print({col:train.stat.corr('Survived',col) 
       for col in ['Pclass','Age','SibSp','Parch','Fare']})

{'Pclass': -0.3384810359610151, 'Age': 0.010539215871285682, 'SibSp': -0.0353224988857356, 'Parch': 0.08162940708348339, 'Fare': 0.2573065223849626}


In [26]:
# Checking which columns have NULL values
print({col:train.where(train[col].isNull()).count()
       for col in train.columns})

{'PassengerId': 0, 'Survived': 0, 'Pclass': 0, 'Name': 0, 'Sex': 0, 'Age': 177, 'SibSp': 0, 'Parch': 0, 'Ticket': 0, 'Fare': 0, 'Cabin': 687, 'Embarked': 2}


In [8]:
# Taking the mean age from the Pandas DF
ageMean = float(train_desc.loc['mean']['Age'])
print(ageMean)

29.69911764705882


In [9]:
# Filling the Age in both train and test datasets
trainFilled = train.na.fill({'Age': ageMean, 'Embarked': 'S'})
testFilled = test.na.fill({'Age': ageMean, 'Embarked': 'S'})

In [27]:
from pyspark.sql import functions as F
train.groupby('Sex','PClass').agg(F.mean('age')).toPandas()#show()

,Sex,PClass,avg(age)
0,male,3,26.507589
1,female,3,21.750000
2,female,1,34.611765
3,female,2,28.722973
4,male,2,30.740707
5,male,1,41.281386


### Step 3
- How to handle categorical features?
    - hint: check the Estimators and Transformers
- Assemble all desired features into a Vector using the VectorAssembler Transformer
- Make sure to end up with a DataFrame with two columns: ***Survived*** and ***vFeatures***

In [11]:
from pyspark.ml.feature import StringIndexer, OneHotEncoder
from pyspark.ml.feature import VectorAssembler
from pyspark.mllib.util import MLUtils

indexer1 = (StringIndexer()
           .setInputCol("Embarked")
           .setOutputCol("nEmbarked")
           .setHandleInvalid('skip'))

indexed1 = indexer1.fit(trainFilled).transform(trainFilled)

In [12]:
indexed1.select('Embarked','nEmbarked').show(20)

+--------+---------+
|Embarked|nEmbarked|
+--------+---------+
|       S|      0.0|
|       C|      1.0|
|       S|      0.0|
|       S|      0.0|
|       S|      0.0|
|       Q|      2.0|
|       S|      0.0|
|       S|      0.0|
|       S|      0.0|
|       C|      1.0|
|       S|      0.0|
|       S|      0.0|
|       S|      0.0|
|       S|      0.0|
|       S|      0.0|
|       S|      0.0|
|       Q|      2.0|
|       S|      0.0|
|       S|      0.0|
|       C|      1.0|
+--------+---------+
only showing top 20 rows



In [13]:
indexer2 = (StringIndexer()
           .setInputCol("Sex")
           .setOutputCol("nSex")
           .setHandleInvalid('skip'))

indexed2 = indexer2.fit(indexed1).transform(indexed1)

In [14]:
encoder1 = OneHotEncoder().setInputCol("nEmbarked").setOutputCol("vEmbarked")
encoded1 = encoder1.transform(indexed2)

In [15]:
encoded1.select('nEmbarked','vEmbarked').show(10)

+---------+-------------+
|nEmbarked|    vEmbarked|
+---------+-------------+
|      0.0|(2,[0],[1.0])|
|      1.0|(2,[1],[1.0])|
|      0.0|(2,[0],[1.0])|
|      0.0|(2,[0],[1.0])|
|      0.0|(2,[0],[1.0])|
|      2.0|    (2,[],[])|
|      0.0|(2,[0],[1.0])|
|      0.0|(2,[0],[1.0])|
|      0.0|(2,[0],[1.0])|
|      1.0|(2,[1],[1.0])|
+---------+-------------+
only showing top 10 rows



In [16]:
encoder2 = OneHotEncoder().setInputCol("nSex").setOutputCol("vSex").setDropLast(False)
encoded2 = encoder2.transform(encoded1)

In [17]:
encoded2.select('Sex', 'nSex','vSex').show(10)

+------+----+-------------+
|   Sex|nSex|         vSex|
+------+----+-------------+
|  male| 0.0|(2,[0],[1.0])|
|female| 1.0|(2,[1],[1.0])|
|female| 1.0|(2,[1],[1.0])|
|female| 1.0|(2,[1],[1.0])|
|  male| 0.0|(2,[0],[1.0])|
|  male| 0.0|(2,[0],[1.0])|
|  male| 0.0|(2,[0],[1.0])|
|  male| 0.0|(2,[0],[1.0])|
|female| 1.0|(2,[1],[1.0])|
|female| 1.0|(2,[1],[1.0])|
+------+----+-------------+
only showing top 10 rows



In [18]:
# Using a VectorAssembler to put together all feature columns
assembler = VectorAssembler(inputCols=['Pclass',
                                       'Age',
                                       'SibSp',
                                       'Parch',
                                       'Fare',
                                       'vSex',
                                       'vEmbarked'], 
                            outputCol='vFeatures')

assembled = assembler.transform(encoded2)

In [19]:
# Keeping only the features and label columns to 
assembled2 = assembled.select("Survived","vFeatures")

In [20]:
assembled2.show(10)

+--------+--------------------+
|Survived|           vFeatures|
+--------+--------------------+
|     0.0|[3.0,22.0,1.0,0.0...|
|     1.0|[1.0,38.0,1.0,0.0...|
|     1.0|[3.0,26.0,0.0,0.0...|
|     1.0|[1.0,35.0,1.0,0.0...|
|     0.0|[3.0,35.0,0.0,0.0...|
|     0.0|(9,[0,1,4,5],[3.0...|
|     0.0|[1.0,54.0,0.0,0.0...|
|     0.0|[3.0,2.0,3.0,1.0,...|
|     1.0|[3.0,27.0,0.0,2.0...|
|     1.0|[2.0,14.0,1.0,0.0...|
+--------+--------------------+
only showing top 10 rows



### Step 4
- Apply a normalization Estimator of your choice to the ***features*** vector obtained in Step 3

In [21]:
from pyspark.ml.feature import StandardScaler

scaler = StandardScaler().setInputCol("vFeatures").setOutputCol("scaledFeat").setWithStd(True).setWithMean(True)
scalerModel = scaler.fit(assembled2)
scaled = scalerModel.transform(assembled2)

In [22]:
scaled.show(10)

+--------+--------------------+--------------------+
|Survived|           vFeatures|          scaledFeat|
+--------+--------------------+--------------------+
|     0.0|[3.0,22.0,1.0,0.0...|[0.82691281652436...|
|     1.0|[1.0,38.0,1.0,0.0...|[-1.5652278312782...|
|     1.0|[3.0,26.0,0.0,0.0...|[0.82691281652436...|
|     1.0|[1.0,35.0,1.0,0.0...|[-1.5652278312782...|
|     0.0|[3.0,35.0,0.0,0.0...|[0.82691281652436...|
|     0.0|(9,[0,1,4,5],[3.0...|[0.82691281652436...|
|     0.0|[1.0,54.0,0.0,0.0...|[-1.5652278312782...|
|     0.0|[3.0,2.0,3.0,1.0,...|[0.82691281652436...|
|     1.0|[3.0,27.0,0.0,2.0...|[0.82691281652436...|
|     1.0|[2.0,14.0,1.0,0.0...|[-0.3691575073769...|
+--------+--------------------+--------------------+
only showing top 10 rows



### Step 5
- Instead of doing transformations on separate steps, put everything together with a Pipeline

In [ ]:
from pyspark.ml.pipeline import Pipeline

### INSERT YOUR CODE HERE

### Step 6
- Train a classifier of your choice (for instance, Random Forest) using your dataset of LabeledPoints
- Make predictions for the training data
- Use the evaluators to find the Area Under ROC and Accuracy of your model
- How is your model performing? Try to tune its parameters

In [ ]:
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.classification import RandomForestClassificationModel
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator

### INSERT YOUR CODE HERE

### Step 7
- Take a look at the test data - use DataFrame's ***createOrReplaceTempView*** method to perform SQL queries over the data
    - hint: check if there are any NULL values in the dataset - if so, handle them
- Apply the transformations to the test data
    - hint: include the model to the pipeline
- Make predictions using the model previously trained and the transformed test data

In [ ]:
### INSERT YOUR CODE HERE

### Step 8

- Load the answers for the ***test*** data
- Combine it with your predictions into a single DataFrame
- Use the evaluator you created on ***Step 6***
- What was your score?

In [ ]:
### INSERT YOUR CODE HERE